In [92]:
import sys
# install dependencies
%pip install numpy
%pip install polars
%pip install defillama2
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import polars as pl
import pandas as pd
from defillama2 import DefiLlama
import matplotlib.pyplot as plt
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
base_dir = '../'
png_dir = os.path.join(base_dir, 'png/volatility')
os.makedirs(png_dir, exist_ok=True)

In [3]:
obj = DefiLlama()

In [19]:
start = '2019-12-18'
end = datetime.now().strftime("%Y-%m-%d")
print(f"Fetching prices from {start} to {end}")
weth = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
dd = {weth: 'ethereum', 'ethereum': 'coingecko'}
raw = obj.get_daily_open_close(dd, start=start, end=end, kind='close')
raw.head()

Fetching prices from 2019-12-18 to 2023-08-22


,ETH,WETH
date,,
2019-12-18,132.741361,133.838204
2019-12-19,128.141042,128.351830
2019-12-20,128.288046,128.216758
2019-12-21,127.060495,126.926953
2019-12-22,132.177470,132.701538


In [20]:
print(
    f"Number of NA values for...\nWETH: {raw.WETH.isna().sum()}\nETH: {raw.ETH.isna().sum()}")

# NA Values
raw.loc[(raw.WETH.isna() == True) | (raw.ETH.isna() == True)]

Number of NA values for...
WETH: 1
ETH: 1


,ETH,WETH
date,,
2022-08-30,NaN,1523.194334
2022-10-11,1280.002846,NaN


In [22]:
# Fill ETH using WETH, or WETH using ETH.... just get 1 dataset
# Using WETH
fillETH = raw.ETH.where(~raw.ETH.isna(), raw.WETH)
raw['Close'] = fillETH
print(
    f"Number of NA values for...\nETH: {raw.Close.isna().sum()}")

# Using ETH
# fillWETH = df.WETH.where(~df.WETH.isna(), df.ETH)
# df['filledWETH'] = fillWETH

Number of NA values for...
ETH: 0


# Finding using normal way of getting standard deviation


In [41]:
# Define helper function to calculate std dev using a rolling window
annualize = np.sqrt(365)


def calculate_vols(df: pd.DataFrame, window_size: int,  ddof: int, deviationFn: callable,) -> pd.DataFrame:
    res = df.copy()
    res['deviation'] = deviationFn(res, window_size)
    res['squared'] = np.square(res['deviation'])
    res['sum'] = res['squared'].rolling(window=window_size).sum()
    res['variance'] = res['sum']/(window_size-ddof)
    res['std-dev'] = np.sqrt(res['variance'])

    res['vol'] = res['std-dev']*annualize
    return res[window_size:]


def calculate_vols_std(df: pd.DataFrame, window_size: int, deviationFn: callable) -> pd.DataFrame:
    res = df.copy()
    res['deviation'] = deviationFn(res, window_size)
    res['std-dev'] = res['deviation'].rolling(window=window_size).std()

    res['vol'] = res['std-dev']*annualize
    return res[window_size:]

In [52]:
# Volatility is measured as the root of sum of squares of movements, not root of sum of squares of deviations from average
def deviationFn(x: pd.DataFrame, window_size: int):
    return x['Close'].diff()/x['Close']


res = calculate_vols(df=raw, window_size=30,  ddof=0, deviationFn=deviationFn)
display(res[30:].head(5))
display(res.tail(5))

,ETH,WETH,Close,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,
2020-02-16,261.301073,260.211974,261.301073,-0.009946,0.000099,0.047715,0.001591,0.039881,0.761928
2020-02-17,267.935666,266.124827,267.935666,0.024762,0.000613,0.047883,0.001596,0.039951,0.763267
2020-02-18,281.945740,282.025591,281.945740,0.049691,0.002469,0.048266,0.001609,0.040111,0.766311
2020-02-19,259.238281,253.285946,259.238281,-0.087593,0.007673,0.055938,0.001865,0.043181,0.824974
2020-02-20,257.988395,256.184578,257.988395,-0.004845,0.000023,0.055714,0.001857,0.043095,0.823319


,ETH,WETH,Close,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,
2023-08-17,1683.640000,1684.19,1683.640000,-0.072801,0.005300,0.007431,0.000248,0.015739,0.300684
2023-08-18,1661.196153,1661.51,1661.196153,-0.013511,0.000183,0.007601,0.000253,0.015917,0.304097
2023-08-19,1668.260000,1669.25,1668.260000,0.004234,0.000018,0.007619,0.000254,0.015936,0.304456
2023-08-20,1685.289419,1686.86,1685.289419,0.010105,0.000102,0.007721,0.000257,0.016042,0.306487
2023-08-21,1667.100000,1668.18,1667.100000,-0.010911,0.000119,0.007566,0.000252,0.015881,0.303404


In [47]:
# Measuring the traditional way
def deviationFn(x: pd.DataFrame, window_size: int):
    x['avg'] = x['Close'].rolling(window=window_size).mean()
    return (x['Close'] - x['avg'])/x['Close']


res = calculate_vols(df=raw, window_size=30, ddof=1, deviationFn=deviationFn)
display(res[30:].head(10))
display(res.tail(5))

,ETH,WETH,Close,avg,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,,
2020-02-16,261.301073,260.211974,261.301073,200.904241,0.231139,0.053425,1.027229,0.035422,0.188207,3.595680
2020-02-17,267.935666,266.124827,267.935666,204.027484,0.238521,0.056892,1.044331,0.036011,0.189767,3.625488
2020-02-18,281.945740,282.025591,281.945740,207.871438,0.262725,0.069025,1.089257,0.037561,0.193806,3.702649
2020-02-19,259.238281,253.285946,259.238281,210.958835,0.186236,0.034684,1.102252,0.038009,0.194958,3.724671
2020-02-20,257.988395,256.184578,257.988395,213.915784,0.170832,0.029183,1.107910,0.038204,0.195458,3.734217
2020-02-21,265.103437,255.451281,265.103437,217.158298,0.180854,0.032708,1.121528,0.038673,0.196655,3.757098
2020-02-22,261.422290,257.101494,261.422290,220.437391,0.156777,0.024579,1.134928,0.039135,0.197827,3.779475
2020-02-23,274.631216,273.572786,274.631216,224.183506,0.183693,0.033743,1.159893,0.039996,0.199991,3.820819
2020-02-24,265.242191,263.148188,265.242191,227.684881,0.141596,0.020050,1.174340,0.040494,0.201232,3.844539


,ETH,WETH,Close,avg,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,,
2023-08-17,1683.640000,1684.19,1683.640000,1848.444810,-0.097886,0.009582,0.018807,0.000649,0.025466,0.486528
2023-08-18,1661.196153,1661.51,1661.196153,1840.792682,-0.108113,0.011688,0.030492,0.001051,0.032426,0.619496
2023-08-19,1668.260000,1669.25,1668.260000,1833.368349,-0.098970,0.009795,0.040274,0.001389,0.037266,0.711970
2023-08-20,1685.289419,1686.86,1685.289419,1826.493663,-0.083786,0.007020,0.047284,0.001630,0.040379,0.771446
2023-08-21,1667.100000,1668.18,1667.100000,1820.038663,-0.091739,0.008416,0.055317,0.001907,0.043675,0.834405


# Finding using log instead...


In [53]:
def deviationFn(x: pd.DataFrame, window_size: int):
    return np.log(x['Close'].shift(-1)/x['Close'])


res = calculate_vols(df=raw, window_size=30, ddof=0, deviationFn=deviationFn)
display(res[30:].head(10))
display(res.tail(5))

,ETH,WETH,Close,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,
2020-02-16,261.301073,260.211974,261.301073,0.025074,0.000629,0.050107,0.001670,0.040869,0.780792
2020-02-17,267.935666,266.124827,267.935666,0.050968,0.002598,0.050710,0.001690,0.041114,0.785473
2020-02-18,281.945740,282.025591,281.945740,-0.083967,0.007050,0.057760,0.001925,0.043879,0.838301
2020-02-19,259.238281,253.285946,259.238281,-0.004833,0.000023,0.057532,0.001918,0.043792,0.836644
2020-02-20,257.988395,256.184578,257.988395,0.027205,0.000740,0.058198,0.001940,0.044045,0.841472
2020-02-21,265.103437,255.451281,265.103437,-0.013983,0.000196,0.057559,0.001919,0.043802,0.836841
2020-02-22,261.422290,257.101494,261.422290,0.049292,0.002430,0.059964,0.001999,0.044708,0.854147
2020-02-23,274.631216,273.572786,274.631216,-0.034786,0.001210,0.061013,0.002034,0.045097,0.861585
2020-02-24,265.242191,263.148188,265.242191,-0.065936,0.004348,0.063297,0.002110,0.045934,0.877558


,ETH,WETH,Close,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,,
2023-08-17,1683.640000,1684.19,1683.640000,-0.013420,0.000180,0.007223,0.000241,0.015516,0.296440
2023-08-18,1661.196153,1661.51,1661.196153,0.004243,0.000018,0.007241,0.000241,0.015536,0.296809
2023-08-19,1668.260000,1669.25,1668.260000,0.010156,0.000103,0.007344,0.000245,0.015646,0.298914
2023-08-20,1685.289419,1686.86,1685.289419,-0.010852,0.000118,0.007192,0.000240,0.015484,0.295817
2023-08-21,1667.100000,1668.18,1667.100000,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
import numpy as np


def calculate_historical_vols(df, sessions_in_year):
    # calculate first log returns using the open
    log_returns = [np.nan]
    # log_returns.append(np.log(df['Close'][0] / df['Open'][0]))
    # calculate all but first log returns using close to close
    for index in range(len(df) - 1):
        log_returns.append(
            np.log(df['Close'][index + 1] / df['Close'][index]))
    df = df.assign(log_returns=log_returns)

    # calculate the 7-day standard deviation and vol
    if len(df) > 6:
        sd_7_day = [np.nan] * 6
        vol_7_day = [np.nan] * 6
        for index in range(len(df) - 6):
            sd = np.std(df['log_returns'][index:index + 6], ddof=1)
            sd_7_day.append(sd)
            vol_7_day.append(sd * np.sqrt(sessions_in_year))
        df = df.assign(sd_7_day=sd_7_day)
        df = df.assign(vol_7_day=vol_7_day)

    # calculate the 30-day standard deviation and vol
    if len(df) > 29:
        sd_30_day = [np.nan] * 29
        vol_30_day = [np.nan] * 29
        for index in range(len(df) - 29):
            sd = np.std(df['log_returns'][index:index + 29], ddof=1)
            sd_30_day.append(sd)
            vol_30_day.append(sd * np.sqrt(sessions_in_year))
        df = df.assign(sd_30_day=sd_30_day)
        df = df.assign(vol_30_day=vol_30_day)

    # calculate the 60-day standard deviation and vol
    if len(df) > 59:
        sd_60_day = [np.nan] * 59
        vol_60_day = [np.nan] * 59

        for index in range(len(df) - 59):
            sd = np.std(df['log_returns'][index:index + 59], ddof=1)
            sd_60_day.append(sd)
            vol_60_day.append(sd * np.sqrt(sessions_in_year))
        df = df.assign(sd_60_day=sd_60_day)
        df = df.assign(vol_60_day=vol_60_day)

    # calculate the 90-day standard deviation and vol
    if len(df) > 89:
        sd_90_day = [np.nan] * 89
        vol_90_day = [np.nan] * 89

        for index in range(len(df) - 89):
            sd = np.std(df['log_returns'][index:index + 89], ddof=1)
            sd_90_day.append(sd)
            vol_90_day.append(sd * np.sqrt(sessions_in_year))
        df = df.assign(sd_90_day=sd_90_day)
        df = df.assign(vol_90_day=vol_90_day)

    # calculate the 180-day standard deviation and vol
    if len(df) > 179:
        sd_180_day = [np.nan] * 179
        vol_180_day = [np.nan] * 179

        for index in range(len(df) - 179):
            sd = np.std(df['log_returns'][index:index + 179], ddof=1)
            sd_180_day.append(sd)
            vol_180_day.append(sd * np.sqrt(sessions_in_year))
        df = df.assign(sd_180_day=sd_180_day)
        df = df.assign(vol_180_day=vol_180_day)

        return df

In [ ]:
sessions_in_year = 365
deribit_df = calculate_historical_vols(raw, sessions_in_year)

# More advanced way


In [56]:
# This way takes very long, so using the simpler way might be better!
start = '2019-12-18'
end = datetime.now().strftime("%Y-%m-%d")
print(f"Fetching prices from {start} to {end}")
weth = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
dd = {weth: 'ethereum', 'ethereum': 'coingecko'}
raw = obj.get_tokens_hist_prices(dd, start=start, end=end, freq='daily')
raw.head()

Fetching prices from 2019-12-18 to 2023-08-22


open                     low                    high   
                   ETH        WETH         ETH        WETH         ETH   
date                                                                     
2019-12-18  121.800690  122.114836  120.460330  118.381428  132.797085  \
2019-12-19  132.741361  133.838204  125.983547  126.412584  132.741361   
2019-12-20  128.141042  128.351830  126.242453  126.217722  128.549793   
2019-12-21  128.288046  128.216758  126.729213  126.912833  128.288046   
2019-12-22  127.060495  126.926953  126.958065  126.926953  132.815763   

                             close                  median               
                  WETH         ETH        WETH         ETH        WETH   
date                                                                     
2019-12-18  132.900943  132.741361  133.838204  123.821413  124.321416  \
2019-12-19  133.838204  128.141042  128.351830  127.287004  127.565486   
2019-12-20  128.467494  128.288046  128.216758  127.275817  127.354211   
2019-12-21  128.216758  127.060495  126.926953  127.248901  127.309585   
2019-12-22  132.593721  132.177470  132.701538  129.298748  129.335190   

                  mean                   std            
                   ETH        WETH       ETH      WETH  
date                                                    
2019-12-18  124.846260  124.971655  3.310591  3.504684  
2019-12-19  127.819247  127.747056  1.589369  1.501943  
2019-12-20  127.493602  127.521503  0.680663  0.677743  
2019-12-21  127.274853  127.336229  0.309593  0.265425  
2019-12-22  129.236901  129.255850  1.894737  1.831112

In [57]:
print(
    f"Number of NA values for...\nWETH: close {raw.close.WETH.isna().sum()} open {raw.open.WETH.isna().sum()}\nETH: close {raw.close.ETH.isna().sum()} open {raw.open.ETH.isna().sum()}")

print(
    f"Number of values where both WETH and ETH are NA...\nclose: {raw.loc[(raw.close.WETH.isna() == True) & (raw.close.ETH.isna() == True)].close.ETH.isna().sum()}\nopen: {raw.loc[(raw.open.WETH.isna() == True) & (raw.open.ETH.isna() == True)].open.ETH.isna().sum()}")
# NA Values
display(raw.loc[(raw.close.WETH.isna() == True) | (raw.close.ETH.isna() == True)
                | (raw.open.WETH.isna() == True) | (raw.open.ETH.isna() == True)])

display(raw.loc[(raw.close.WETH.isna() == True) & (raw.close.ETH.isna() == True)
                | (raw.open.WETH.isna() == True) & (raw.open.ETH.isna() == True)])

Number of NA values for...
WETH: close 28 open 27
ETH: close 14 open 13
Number of values where both WETH and ETH are NA...
close: 3
open: 2


open                       low                      high   
                    ETH         WETH          ETH         WETH          ETH   
date                                                                          
2020-10-12   374.497015   373.453271   366.723229   366.988604   392.263524  \
2020-10-13   386.881591          NaN   377.363153   375.724461   386.881591   
2020-10-14   381.078881          NaN   374.243575   375.107106   386.579840   
2020-10-17   365.659373   365.054770   364.424889   365.015418   369.305968   
2020-10-18   368.646795          NaN   368.443725   367.492469   377.160891   
...                 ...          ...          ...          ...          ...   
2022-10-07  1351.427369  1353.846562  1323.379132  1321.499713  1358.003832   
2022-10-08          NaN          NaN  1311.154367  1309.852623  1330.834875   
2022-10-11  1291.241113  1292.337211  1275.610631  1273.806344  1296.651783   
2022-10-12  1280.002846          NaN  1280.002846  1291.370000  1303.367590   
2023-08-21  1685.289419  1686.860000  1658.710000  1658.600000  1685.289419   

                               close                    median                
                   WETH          ETH         WETH          ETH         WETH   
date                                                                          
2020-10-12   393.120462   386.881591          NaN   374.335210   374.875383  \
2020-10-13   386.670875   381.078881          NaN   382.799816   382.986208   
2020-10-14   385.669429   379.566363   377.688930   381.210281   381.573314   
2020-10-17   368.356910   368.646795          NaN   367.760352   367.643553   
2020-10-18   376.943190   377.973294   375.910136   373.793888   373.646117   
...                 ...          ...          ...          ...          ...   
2022-10-07  1358.015027          NaN          NaN  1354.929462  1353.609650   
2022-10-08  1330.400826  1315.913794  1315.493480  1327.466408  1326.213511   
2022-10-11  1292.337211  1280.002846          NaN  1282.038478  1279.598888   
2022-10-12  1297.896295  1294.330000  1292.480000  1295.645035  1296.835194   
2023-08-21  1686.860000          NaN          NaN  1672.223643  1671.160000   

                   mean                     std             
                    ETH         WETH        ETH       WETH  
date                                                        
2020-10-12   378.723311   379.054588   8.098492   8.034617  
2020-10-13   382.096989   381.980461   2.851500   2.881310  
2020-10-14   380.129492   380.451072   3.310587   3.146544  
2020-10-17   367.469256   367.454443   1.013141   0.838477  
2020-10-18   373.026450   372.929500   2.926752   2.881727  
...                 ...          ...        ...        ...  
2022-10-07  1346.738002  1346.947829  12.880362  12.951339  
2022-10-08  1325.901438  1325.268075   5.421185   5.444821  
2022-10-11  1282.935275  1281.041325   5.950868   5.206521  
2022-10-12  1294.115886  1295.658096   6.307536   2.480338  
2023-08-21  1672.559737  1672.411739   6.704814   6.796251  

[69 rows x 14 columns]

open                       low                      high   
                    ETH         WETH          ETH         WETH          ETH   
date                                                                          
2022-05-19  1915.177123  1918.261509  1915.177123  1918.261509  2024.421850  \
2022-05-20          NaN          NaN  1932.876151  1940.398431  2057.341227   
2022-10-07  1351.427369  1353.846562  1323.379132  1321.499713  1358.003832   
2022-10-08          NaN          NaN  1311.154367  1309.852623  1330.834875   
2023-08-21  1685.289419  1686.860000  1658.710000  1658.600000  1685.289419   

                               close                   median                
                   WETH          ETH        WETH          ETH         WETH   
date                                                                         
2022-05-19  2031.884877          NaN         NaN  1957.579314  1956.579668  \
2022-05-20  2057.736878  1963.990940  1962.77437  1982.675520  2003.207273   
2022-10-07  1358.015027          NaN         NaN  1354.929462  1353.609650   
2022-10-08  1330.400826  1315.913794  1315.49348  1327.466408  1326.213511   
2023-08-21  1686.860000          NaN         NaN  1672.223643  1671.160000   

                   mean                     std             
                    ETH         WETH        ETH       WETH  
date                                                        
2022-05-19  1964.709083  1965.314711  33.153897  33.740643  
2022-05-20  1994.645260  1998.873997  43.932225  42.345934  
2022-10-07  1346.738002  1346.947829  12.880362  12.951339  
2022-10-08  1325.901438  1325.268075   5.421185   5.444821  
2023-08-21  1672.559737  1672.411739   6.704814   6.796251

In [58]:
# Assumption: WETH and ETH peg, thus we can use 1 price for the other "theoretically-ish"

# Fill ETH using WETH, or WETH using ETH.... just get 1 dataset
# Using WETH
df = pd.DataFrame()
close_price = raw.close.ETH.where(~raw.close.ETH.isna(), raw.close.WETH)
df['Close'] = close_price
open_price = raw.open.ETH.where(~raw.open.ETH.isna(), raw.open.WETH)
df['Open'] = open_price

df['Close'] = df['Close'].interpolate(method='linear')
df['Open'] = df['Open'].interpolate(method='linear')
print(
    f"Number of NA values for...\nETH: close {df['Close'].isna().sum()} open {df['Open'].isna().sum()}")
display(df.head(10))

Number of NA values for...
ETH: close 0 open 0


,Close,Open
date,,
2019-12-18,132.741361,121.800690
2019-12-19,128.141042,132.741361
2019-12-20,128.288046,128.141042
2019-12-21,127.060495,128.288046
2019-12-22,132.177470,127.060495
2019-12-23,128.099756,132.177470
2019-12-24,127.944089,128.099756
2019-12-25,125.125175,127.944089
2019-12-26,125.347398,125.125175


In [59]:
def deviationFn(x: pd.DataFrame, window_size: int):
    return np.log(x['Close']/x['Close'].shift(1))


res = calculate_vols(df=df, window_size=30, ddof=0, deviationFn=deviationFn)
display(res.head(10))
display(res.tail(10))

,Close,Open,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,
2020-01-17,170.561769,163.805901,0.040415,1.633398e-03,0.042203,0.001407,0.037507,0.716567
2020-01-18,174.238373,170.561769,0.021327,4.548329e-04,0.041414,0.001380,0.037154,0.709835
2020-01-19,166.627106,174.238373,-0.044666,1.995043e-03,0.043407,0.001447,0.038038,0.726721
2020-01-20,166.616376,166.627106,-0.000064,4.147031e-09,0.043315,0.001444,0.037998,0.725946
2020-01-21,169.279936,166.616376,0.015860,2.515317e-04,0.042008,0.001400,0.037420,0.714907
2020-01-22,167.828012,169.279936,-0.008614,7.420193e-05,0.041100,0.001370,0.037013,0.707141
2020-01-23,163.049502,167.828012,-0.028886,8.343933e-04,0.041933,0.001398,0.037387,0.714270
2020-01-24,162.247778,163.049502,-0.004929,2.429692e-05,0.041461,0.001382,0.037176,0.710238
2020-01-25,160.200929,162.247778,-0.012696,1.611840e-04,0.041619,0.001387,0.037246,0.711591


,Close,Open,deviation,squared,sum,variance,std-dev,vol
date,,,,,,,,
2023-08-12,1848.700000,1847.560000,0.000617,3.804913e-07,0.003145,0.000105,0.010239,0.195613
2023-08-13,1839.370000,1848.700000,-0.005060,2.559923e-05,0.002014,0.000067,0.008193,0.156536
2023-08-14,1844.114824,1839.370000,0.002576,6.637169e-06,0.002013,0.000067,0.008191,0.156497
2023-08-15,1826.439480,1844.114824,-0.009631,9.275539e-05,0.002088,0.000070,0.008343,0.159399
2023-08-16,1806.210000,1826.439480,-0.011138,1.240485e-04,0.002178,0.000073,0.008520,0.162773
2023-08-17,1683.640000,1806.210000,-0.070273,4.938240e-03,0.007055,0.000235,0.015336,0.292987
2023-08-18,1661.196153,1683.640000,-0.013420,1.801017e-04,0.007223,0.000241,0.015516,0.296440
2023-08-19,1669.126414,1661.196153,0.004762,2.268109e-05,0.007245,0.000242,0.015541,0.296905
2023-08-20,1685.289419,1669.126414,0.009637,9.287036e-05,0.007338,0.000245,0.015640,0.298800


In [60]:
res = calculate_historical_vols(df, 365)
display(res[25:].head(10))
display(res.tail(20))

,Close,Open,log_returns,sd_7_day,vol_7_day,sd_30_day,vol_30_day,sd_60_day,vol_60_day,sd_90_day,vol_90_day,sd_180_day,vol_180_day
date,,,,,,,,,,,,,
2020-01-12,145.219436,142.182333,0.021136,0.037089,0.708582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-13,143.587969,145.219436,-0.011298,0.027889,0.532816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-14,165.991439,143.587969,0.144988,0.028284,0.540357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-15,166.264210,165.991439,0.001642,0.063127,1.206047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-16,163.805901,166.264210,-0.014896,0.060603,1.157820,0.037779,0.721775,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-17,170.561769,163.805901,0.040415,0.062474,1.193571,0.037342,0.713425,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-18,174.238373,170.561769,0.021327,0.059890,1.144197,0.036907,0.705101,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-19,166.627106,174.238373,-0.044666,0.059884,1.144086,0.036927,0.705485,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-20,166.616376,166.627106,-0.000064,0.065786,1.256832,0.038164,0.729113,NaN,NaN,NaN,NaN,NaN,NaN


,Close,Open,log_returns,sd_7_day,vol_7_day,sd_30_day,vol_30_day,sd_60_day,vol_60_day,sd_90_day,vol_90_day,sd_180_day,vol_180_day
date,,,,,,,,,,,,,
2023-08-02,1838.360000,1870.500000,-0.017332,0.007766,0.148374,0.017368,0.331817,0.021691,0.414397,0.021112,0.403339,0.024981,0.477261
2023-08-03,1834.920000,1838.360000,-0.001873,0.010398,0.198644,0.017566,0.335595,0.021804,0.416569,0.020177,0.385488,0.025017,0.477943
2023-08-04,1828.660000,1834.920000,-0.003417,0.009143,0.174680,0.017437,0.333136,0.020992,0.401046,0.019549,0.373485,0.024962,0.476908
2023-08-05,1835.440000,1828.660000,0.003701,0.008510,0.162577,0.016506,0.315347,0.020343,0.388661,0.019545,0.373404,0.024950,0.476678
2023-08-06,1827.060000,1835.440000,-0.004576,0.008604,0.164381,0.016431,0.313907,0.020033,0.382730,0.019406,0.370756,0.024820,0.474195
2023-08-07,1827.350000,1827.060000,0.000159,0.008646,0.165189,0.016442,0.314131,0.020018,0.382442,0.019412,0.370867,0.024802,0.473835
2023-08-08,1855.640000,1827.350000,0.015363,0.007204,0.137625,0.016442,0.314127,0.020013,0.382352,0.019408,0.370789,0.024299,0.464229
2023-08-09,1854.980000,1855.640000,-0.000356,0.007366,0.140724,0.016609,0.317322,0.019104,0.364975,0.019299,0.368700,0.024269,0.463664
2023-08-10,1850.980000,1854.980000,-0.002159,0.007250,0.138502,0.016609,0.317320,0.019102,0.364947,0.019289,0.368523,0.024239,0.463077
